In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import glob

In [2]:
#Clean Data

In [38]:
years = [2014,2010,2006,2002, 1998]#,1998,1994,1990]

In [56]:
directory = "/Users/anjian/Dropbox (MIT)/Courses/Fall 2017/6.439/Class Project/Project Data/"

In [57]:
#pd.DataFrame.from_csv(directory+"2014.csv")

In [70]:
for year in years:
    print("Loading year %s"%year)

    data = pd.DataFrame.from_csv(directory+"%s.csv"%year)
    for i in range(len(data["Matches Played"])):
        if np.isnan(data["Matches Played"].iloc[i]):
            data["Matches Played"].iloc[i] = data["Matches Played.1"].iloc[i]

            
        data["Yellow_Per_Game_Avg"] = data["Yellow Card"]/data["Matches Played"]
        data["YellowRed_Per_Game_Avg"] = data["Second yellow card and red card"]/data["Matches Played"]
        data["Red_Per_Game_Avg"] = data["Red Cards"]/data["Matches Played"]

        data["Goal_Per_Game_Avg"] = data["Goals for"]/data["Matches Played"]
        data["Goal_Against_Per_Game_Avg"] = data["Goals Against"]/data["Matches Played"]
        data["PenGoal_Per_Game_Avg"] = data["Penalty goal"]/data["Matches Played"]

        data.to_csv("./%s.csv"%year)

Loading year 2014
Loading year 2010
Loading year 2006
Loading year 2002
Loading year 1998


In [71]:
data.columns

Index(['number', 'madeSF', 'Teams.1', 'Goals for', 'Goals Against',
       'Penalty goal', 'Average Goals For', 'Matches Played', 'number.1',
       'Teams.1.1', 'Yellow Card', 'Second yellow card and red card',
       'Red Cards', 'Matches Played.1', 'Yellow_Per_Game_Avg',
       'YellowRed_Per_Game_Avg', 'Red_Per_Game_Avg', 'Goal_Per_Game_Avg',
       'Goal_Against_Per_Game_Avg', 'PenGoal_Per_Game_Avg', 'FIFA rank',
       'seed', 'host', 'stars', 'cohesion', 'cohesion sans 1', 'dist'],
      dtype='object')

In [72]:
features_types = ["Matches Played",
                  'Yellow_Per_Game_Avg', 
                  'YellowRed_Per_Game_Avg',
                  'Red_Per_Game_Avg', 
                  'Goal_Per_Game_Avg', 
                  'Goal_Against_Per_Game_Avg',
                  'PenGoal_Per_Game_Avg']

features_types_current = ['FIFA rank', 
                  'seed', 
                  'host', 
                  'stars', 
                  'cohesion',
                  'cohesion sans 1', 
                  'dist']

In [73]:
match_data =  pd.DataFrame.from_csv("/Users/anjian/Dropbox (MIT)/Courses/Fall 2017/6.439/Class Project/Project Data/all_match_outcomes.csv")

In [74]:
feature_matrix = pd.DataFrame()

for y in range(len(years)-1):
    current_year = years[y]
    last_year = years[y+1]
    
    #Grab Last World Cup's Performance Metrics of Country 1 ------
    perf_data = pd.DataFrame.from_csv("./%s.csv"%last_year)
    perf_data = perf_data[features_types]
    perf_data.columns = [x+"_C1" for x in perf_data.columns]
    perf_data = perf_data.reset_index()
    perf_data = perf_data.rename(columns = {'index':'Country1'})
    features_of_this_year = match_data[match_data['Year'] == current_year]
    features_of_this_year = features_of_this_year.merge(perf_data, how='left', on="Country1")
    
    #Grab Current Ranking Features of Country 1 ------
    perf_data = pd.DataFrame.from_csv("./%s.csv"%current_year)
    perf_data = perf_data[features_types_current]
    perf_data.columns = [x+"_C1" for x in perf_data.columns]
    perf_data = perf_data.reset_index()
    perf_data = perf_data.rename(columns = {'index':'Country1'})
    features_of_this_year = features_of_this_year.merge(perf_data, how='left', on="Country1")
    
    #Grab Last World Cup's Performance Metrics of Country 2
    
    perf_data = pd.DataFrame.from_csv("./%s.csv"%last_year)
    perf_data = perf_data[features_types]
    perf_data.columns = [x+"_C2" for x in perf_data.columns]
    perf_data = perf_data.reset_index()
    perf_data = perf_data.rename(columns = {'index':'Country2'})
    features_of_this_year = features_of_this_year.merge(perf_data, how='left', on="Country2")
    
    #Grab Current Ranking Features of Country 2 ------
    perf_data = pd.DataFrame.from_csv("./%s.csv"%current_year)
    perf_data = perf_data[features_types_current]
    perf_data.columns = [x+"_C2" for x in perf_data.columns]
    perf_data = perf_data.reset_index()
    perf_data = perf_data.rename(columns = {'index':'Country2'})
    features_of_this_year = features_of_this_year.merge(perf_data, how='left', on="Country2")
    
    #Combine to Feature Matrix
    feature_matrix = pd.concat([feature_matrix,features_of_this_year], axis=0, ignore_index=True)
    
    print(len(features_of_this_year))


64
64
64
64


In [75]:
feature_matrix.to_csv("./feature_matrix.csv")

In [113]:
#Some Countries did not play in previous year, need to just zero them out
feature_matrix = feature_matrix.fillna(0)
